<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Langchain_NL2SQL_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.aidemos.com/

[Discover Best AI Tools with Video Demos](https://www.aidemos.com/)



1. Building a basic NL2SQL model
2. Adding few-shot examples
3. Dynamic few-shot example selection
4. Dynamic relevant table selection (Large DB Isssue)
5. Customizing prompts
6. Adding memory to the chatbot so that it answers follow-up questions related to the database.

https://github.com/PradipNichite/Youtube-Tutorials/blob/main/Langchain_NL2SQL_2024.ipynb


In [ ]:
#!pip install langchain_openai langchain_community langchain langchain-experimental pymysql chromadb -q


In [ ]:
#pip install -U langsmith


https://www.mysqltutorial.org/getting-started-with-mysql/mysql-sample-database/

In [ ]:
#%pip install python-dotenv


In [ ]:
import os
import pandas as pd
from typing import List
from langchain_openai import AzureChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.memory import ChatMessageHistory


In [ ]:

#import load_dotenv
#load_dotenv()
db_user = os.getenv("MySql_db_user")
db_password = os.getenv("MySql_db_password")
db_host = os.getenv("MySql_db_host")
db_name = os.getenv("MySql_db_name")
db_port = os.getenv("MySql_db_port")

LANGCHAIN_TRACING_V2 = "false"
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]

# Load config values
openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT") 
openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION") 
azure_deployment="gpt-4-32k"
openai_api_key = os.getenv("AZURE_OPENAI_KEY") 
openai_api_type="azure"


###Building a basic NL2SQL model

In [ ]:

# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")


In [ ]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)


In [ ]:


# Create an instance of chat llm
llm = AzureChatOpenAI(
    azure_endpoint=openai_api_base,
    openai_api_version=openai_api_version,
    azure_deployment=azure_deployment,
    openai_api_key=openai_api_key,
    openai_api_type=openai_api_type,
)


In [ ]:

#llm = ChatOpenAI(model="gpt-4-32k", temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "what is price of `1968 Ford Mustang`"})
# "what is price of `1968 Ford Mustang`"
print(query)


In [ ]:

execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)


In [ ]:
chain = generate_query | execute_query
chain.invoke({"question": "How many orders are there"})


In [ ]:
chain.get_prompts()[0].pretty_print()


In [ ]:


answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

chain.invoke({"question": "How many orders are there"})


###Adding few-shot examples

In [ ]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
    {
        "input": "Show product details for products in the 'Motorcycles' product line.",
        "query": "SELECT * FROM products WHERE productLine = 'Motorcycles';"
    },
    {
        "input": "Retrieve the names of employees who report to employee number 1002.",
        "query": "SELECT firstName, lastName FROM employees WHERE reportsTo = 1002;"
    },
    {
        "input": "List all products with a stock quantity less than 7000.",
        "query": "SELECT productName, quantityInStock FROM products WHERE quantityInStock < 7000;"
    },
    {
     'input':"what is price of `1968 Ford Mustang`",
     "query": "SELECT `buyPrice`, `MSRP` FROM products  WHERE `productName` = '1968 Ford Mustang' LIMIT 1;"
    }
]


In [ ]:


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # input_variables=["input","top_k"],
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input1="How many products are there?"))


###Dynamic few-shot example selection

In [ ]:

search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_AI_SEARCH_KEY")
index = "vector-1713531272622"
# This is the embedding class used to produce embeddings which are used to measure semantic similarity.
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version=openai_api_version,
    azure_endpoint=openai_api_base,
    api_key=openai_api_key,
)

vectorstore = Chroma()
# vectorstore = AzureSearch(
#     azure_search_endpoint=search_endpoint,
#     azure_search_key=search_key,
#     index_name=index,
#     embedding_function=embeddings.embed_query,
# )
#vectorstore.delete("sqlsearchindex")

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    vectorstore,
    k=2,
    input_keys=["input"],
)
example_selector.select_examples({"input": "how many employees we have?"})
# example_selector.select_examples({"input": "How many employees?"})


In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="How many products are there?"))


Customizing prompts

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info"))


In [ ]:
generate_query = create_sql_query_chain(llm, db,final_prompt)
chain = (
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many csutomers with credit limit more than 50000"})


###Dynamic relevant table selection

In [ ]:


def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("database_table_descriptions.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name:" + row['Table'] + "\n" + "Table Description:" + row['Description'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)


In [ ]:
table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "give me details of customer and their order count"})
tables


In [ ]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
select_table.invoke({"question": "give me details of customer and their order count"})


In [ ]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many cutomers with order count more than 5"})


In [ ]:
chain.invoke({"question": "Can you list their names?"})


###Adding memory to the chatbot so that it answers follow-up questions related to the database.






In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info",messages=[]))


In [ ]:

history = ChatMessageHistory()

generate_query = create_sql_query_chain(llm, db,final_prompt)

chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)


In [ ]:
question = "How many cutomers with order count more than 5"
response = chain.invoke({"question": question,"messages":history.messages})
response


In [ ]:
history.add_user_message(question)
history.add_ai_message(response)


In [ ]:
history.messages


In [ ]:
response = chain.invoke({"question": "Can you list there names?","messages":history.messages})
response
